# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *


In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=jacob password=chani")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
filepath = song_files[0]

In [6]:
filepath

'/home/jacob/Documents/project_one/data/song_data/A/A/A/TRAAAPK128E0786D96.json'

In [7]:
df = pd.read_json(filepath, typ="series")
df

num_songs                                     1
artist_id                    AR10USD1187B99F3F1
artist_latitude                            None
artist_longitude                           None
artist_location     Burlington, Ontario, Canada
artist_name               Tweeterfriendly Music
song_id                      SOHKNRJ12A6701D1F8
title                              Drop of Rain
duration                                 189.57
year                                          0
dtype: object

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_values = df[['song_id', 'title', 'artist_id', 'year', 'duration']]
song_data = song_values.values
song_data

array(['SOHKNRJ12A6701D1F8', 'Drop of Rain', 'AR10USD1187B99F3F1', 0,
       189.57016], dtype=object)

In [9]:
print(song_table_insert, tuple(song_data))


INSERT INTO songs (song_id, title, artist_id, year, duration) 
VALUES (%s, %s, %s, %s, %s) on conflict do nothing 
 ('SOHKNRJ12A6701D1F8', 'Drop of Rain', 'AR10USD1187B99F3F1', 0, 189.57016)


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [10]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [11]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
artist_data

artist_id                    AR10USD1187B99F3F1
artist_name               Tweeterfriendly Music
artist_location     Burlington, Ontario, Canada
artist_latitude                            None
artist_longitude                           None
dtype: object

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [12]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [13]:
log_files = get_files('data/log_data')
log_files.sort()
log_files

['/home/jacob/Documents/project_one/data/log_data/2018/11/2018-11-01-events.json',
 '/home/jacob/Documents/project_one/data/log_data/2018/11/2018-11-02-events.json',
 '/home/jacob/Documents/project_one/data/log_data/2018/11/2018-11-03-events.json',
 '/home/jacob/Documents/project_one/data/log_data/2018/11/2018-11-04-events.json',
 '/home/jacob/Documents/project_one/data/log_data/2018/11/2018-11-05-events.json',
 '/home/jacob/Documents/project_one/data/log_data/2018/11/2018-11-06-events.json',
 '/home/jacob/Documents/project_one/data/log_data/2018/11/2018-11-07-events.json',
 '/home/jacob/Documents/project_one/data/log_data/2018/11/2018-11-08-events.json',
 '/home/jacob/Documents/project_one/data/log_data/2018/11/2018-11-09-events.json',
 '/home/jacob/Documents/project_one/data/log_data/2018/11/2018-11-10-events.json',
 '/home/jacob/Documents/project_one/data/log_data/2018/11/2018-11-11-events.json',
 '/home/jacob/Documents/project_one/data/log_data/2018/11/2018-11-12-events.json',
 '/h

In [31]:
filepath = log_files[1]
filepath

'/home/jacob/Documents/project_one/data/log_data/2018/11/2018-11-02-events.json'

In [32]:
df = pd.read_json(filepath, lines=True)
df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,N.E.R.D. FEATURING MALICE,Logged In,Jayden,M,0,Fox,288.99220,free,"New Orleans-Metairie, LA",PUT,NextSong,1541033612796,184,Am I High (Feat. Malice),200,1541121934796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",101
1,None,Logged In,Stefany,F,0,White,NaN,free,"Lubbock, TX",GET,Home,1540708070796,82,None,200,1541122176796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
2,Death Cab for Cutie,Logged In,Stefany,F,1,White,216.42404,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,A Lack Of Color (Album Version),200,1541122241796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
3,Tracy Gang Pussy,Logged In,Stefany,F,2,White,221.33506,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,I Have A Wish,200,1541122457796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
4,Skillet,Logged In,Kevin,M,0,Arellano,178.02404,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1540006905796,153,Monster (Album Version),200,1541126568796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,OutKast,Logged In,Ayleen,F,3,Wise,227.52608,free,"Columbia, SC",PUT,NextSong,1541085793796,70,Ova Da Wudz,200,1541184339796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_1 like...",71
167,Bloodhound Gang,Logged In,Kaleb,M,0,Cook,260.20526,free,"Yuba City, CA",PUT,NextSong,1540679673796,53,Uhn Tiss Uhn Tiss Uhn Tiss,200,1541187675796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,54
168,Rammstein,Logged In,Isaac,M,0,Valdez,228.46649,free,"Saginaw, MI",PUT,NextSong,1541078099796,112,Adios,200,1541191023796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,3
169,Gershon Kingsley,Logged In,Isaac,M,1,Valdez,146.57261,free,"Saginaw, MI",PUT,NextSong,1541078099796,112,Pop Corn,200,1541191251796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,3


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [33]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [34]:
df = df[df['page'] == 'NextSong']
df.loc[:, 'ts'] = pd.to_datetime(df['ts'], unit='ms')
df

/home/jacob/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,N.E.R.D. FEATURING MALICE,Logged In,Jayden,M,0,Fox,288.99220,free,"New Orleans-Metairie, LA",PUT,NextSong,1541033612796,184,Am I High (Feat. Malice),200,2018-11-02 01:25:34.796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",101
2,Death Cab for Cutie,Logged In,Stefany,F,1,White,216.42404,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,A Lack Of Color (Album Version),200,2018-11-02 01:30:41.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
3,Tracy Gang Pussy,Logged In,Stefany,F,2,White,221.33506,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,I Have A Wish,200,2018-11-02 01:34:17.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
4,Skillet,Logged In,Kevin,M,0,Arellano,178.02404,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1540006905796,153,Monster (Album Version),200,2018-11-02 02:42:48.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
5,Dance Gavin Dance,Logged In,Marina,F,0,Sutton,218.46159,free,"Salinas, CA",PUT,NextSong,1541064343796,47,Uneasy Hearts Weigh The Most,200,2018-11-02 03:05:57.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,OutKast,Logged In,Ayleen,F,3,Wise,227.52608,free,"Columbia, SC",PUT,NextSong,1541085793796,70,Ova Da Wudz,200,2018-11-02 18:45:39.796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_1 like...",71
167,Bloodhound Gang,Logged In,Kaleb,M,0,Cook,260.20526,free,"Yuba City, CA",PUT,NextSong,1540679673796,53,Uhn Tiss Uhn Tiss Uhn Tiss,200,2018-11-02 19:41:15.796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,54
168,Rammstein,Logged In,Isaac,M,0,Valdez,228.46649,free,"Saginaw, MI",PUT,NextSong,1541078099796,112,Adios,200,2018-11-02 20:37:03.796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,3
169,Gershon Kingsley,Logged In,Isaac,M,1,Valdez,146.57261,free,"Saginaw, MI",PUT,NextSong,1541078099796,112,Pop Corn,200,2018-11-02 20:40:51.796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,3


In [35]:
df.loc[:, 'weekday'] = df['ts'].dt.day_name()
df.loc[:, 'weekday']

/home/jacob/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/jacob/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


0      Friday
2      Friday
3      Friday
4      Friday
5      Friday
        ...  
166    Friday
167    Friday
168    Friday
169    Friday
170    Friday
Name: weekday, Length: 155, dtype: object

In [36]:
df.loc[:, 'hour'] = df['ts'].dt.hour
df.loc[:, 'day'] = df['ts'].dt.day
df.loc[:, 'week'] = df['ts'].dt.isocalendar().week
df.loc[:, 'month'] = df['ts'].dt.month
df.loc[:, 'year'] = df['ts'].dt.year
df.loc[:, ['hour', 'day', 'week', 'month', 'year']]

,hour,day,week,month,year
0,1,2,44,11,2018
2,1,2,44,11,2018
3,1,2,44,11,2018
4,2,2,44,11,2018
5,3,2,44,11,2018
...,...,...,...,...,...
166,18,2,44,11,2018
167,19,2,44,11,2018
168,20,2,44,11,2018
169,20,2,44,11,2018


In [44]:
from sql_queries import *
df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,status,ts,userAgent,userId,weekday,hour,day,week,month,year
0,N.E.R.D. FEATURING MALICE,Logged In,Jayden,M,0,Fox,288.99220,free,"New Orleans-Metairie, LA",PUT,...,200,2018-11-02 01:25:34.796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",101,Friday,1,2,44,11,2018
2,Death Cab for Cutie,Logged In,Stefany,F,1,White,216.42404,free,"Lubbock, TX",PUT,...,200,2018-11-02 01:30:41.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83,Friday,1,2,44,11,2018
3,Tracy Gang Pussy,Logged In,Stefany,F,2,White,221.33506,free,"Lubbock, TX",PUT,...,200,2018-11-02 01:34:17.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83,Friday,1,2,44,11,2018
4,Skillet,Logged In,Kevin,M,0,Arellano,178.02404,free,"Harrisburg-Carlisle, PA",PUT,...,200,2018-11-02 02:42:48.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66,Friday,2,2,44,11,2018
5,Dance Gavin Dance,Logged In,Marina,F,0,Sutton,218.46159,free,"Salinas, CA",PUT,...,200,2018-11-02 03:05:57.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",48,Friday,3,2,44,11,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,OutKast,Logged In,Ayleen,F,3,Wise,227.52608,free,"Columbia, SC",PUT,...,200,2018-11-02 18:45:39.796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_1 like...",71,Friday,18,2,44,11,2018
167,Bloodhound Gang,Logged In,Kaleb,M,0,Cook,260.20526,free,"Yuba City, CA",PUT,...,200,2018-11-02 19:41:15.796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,54,Friday,19,2,44,11,2018
168,Rammstein,Logged In,Isaac,M,0,Valdez,228.46649,free,"Saginaw, MI",PUT,...,200,2018-11-02 20:37:03.796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,3,Friday,20,2,44,11,2018
169,Gershon Kingsley,Logged In,Isaac,M,1,Valdez,146.57261,free,"Saginaw, MI",PUT,...,200,2018-11-02 20:40:51.796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,3,Friday,20,2,44,11,2018


In [38]:
df['artist']

0      N.E.R.D. FEATURING MALICE
2            Death Cab for Cutie
3               Tracy Gang Pussy
4                        Skillet
5              Dance Gavin Dance
                 ...            
166                      OutKast
167              Bloodhound Gang
168                    Rammstein
169             Gershon Kingsley
170                    The Rakes
Name: artist, Length: 155, dtype: object

In [39]:
time_columns = ['ts', 'hour', 'day', 'week', 'month', 'year', 'weekday']
time_columns

['ts', 'hour', 'day', 'week', 'month', 'year', 'weekday']

In [40]:
# time_data = ()
# column_labels = ()

In [41]:
time_df = df.loc[:, time_columns]
time_df.head()

,ts,hour,day,week,month,year,weekday
0,2018-11-02 01:25:34.796,1,2,44,11,2018,Friday
2,2018-11-02 01:30:41.796,1,2,44,11,2018,Friday
3,2018-11-02 01:34:17.796,1,2,44,11,2018,Friday
4,2018-11-02 02:42:48.796,2,2,44,11,2018,Friday
5,2018-11-02 03:05:57.796,3,2,44,11,2018,Friday


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [42]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [43]:
user_df = df.loc[:, ['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df

,userId,firstName,lastName,gender,level
0,101,Jayden,Fox,M,free
2,83,Stefany,White,F,free
3,83,Stefany,White,F,free
4,66,Kevin,Arellano,M,free
5,48,Marina,Sutton,F,free
...,...,...,...,...,...
166,71,Ayleen,Wise,F,free
167,54,Kaleb,Cook,M,free
168,3,Isaac,Valdez,M,free
169,3,Isaac,Valdez,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [27]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row.values)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [28]:
for index, row in df.iterrows():
    print(songplay_table_insert, row.song, row.artist, row.length)
    

    # get songid and artistid from song and artist tables
    print(song_select, (row.song, row.artist, row.length))
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    print(results)

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()


insert into songplays (song_id, start_time, user_id, user_agent) 
VALUES (%s)
 You Gotta Be Des'ree 246.30812

select s.song_id, s.artist_id, a.name from songs as s 
join artists as a on (a.artist_id = s.artist_id) 
where s.title=%s and a.name=%s and s.duration=%s
 ('You Gotta Be', "Des'ree", 246.30812)
None


IndexError: tuple index out of range

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.